In [1]:
import boto3

In [ ]:
# Create S3 bucket
s3 = boto3.client('s3', region_name = 'us-east-1')
bucket = s3.create_bucket(Bucket = 'wos')


In [ ]:
# For now: Read XML from local system
# TODO: Use Lambda func to scrape XML from lib website and write to S3
xml = open(, 'rb')


In [ ]:
# For now: Write XML to S3 bucket
s3.Bucket('wos').put_object(
                Key = None, # File name
                Body = xml) # XML file